# Prophet - Predecir el valor de activos de Yahoo Finances


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd> (hay que ejecutarlo dos veces).
</div>
<hr/>


In [56]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [57]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format         #dos decimales


Pasos para predecir valores
(guía: https://facebook.github.io/prophet/docs/quick_start.html)
```
1. Inicializar modelo:
    nombre_variable = Modelo()
2. Búsqueda del mejor hiperparámetro
    lo hace Prophet por su cuenta 
3. Entrenar / Ajustar:
    nombre_variable.fit()
4. Predecir valores futuros
    nombre_variable.predict(futuro)
```



<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [108]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-03-15'
start_date = '2016-01-01'       # más pronto = más datos
# Descargar el dataframe
sp_df = yf.download('^GSPC',start_date, today).reset_index()

sp_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1806,2023-03-08,"$3,987.55","$4,000.41","$3,969.76","$3,992.01","$3,992.01",3535570000
1807,2023-03-09,"$3,998.66","$4,017.81","$3,908.70","$3,918.32","$3,918.32",4445260000
1808,2023-03-10,"$3,912.77","$3,934.05","$3,846.32","$3,861.59","$3,861.59",5518190000
1809,2023-03-13,"$3,835.12","$3,905.05","$3,808.86","$3,855.76","$3,855.76",6558020000
1810,2023-03-14,"$3,894.01","$3,937.29","$3,873.63","$3,919.29","$3,919.29",5665870000


In [109]:
# Validamos que no hay datos vacíos
sp_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [110]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = sp_df[["Date", "Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds" y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)

In [111]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [112]:
df.tail()

,ds,y
1806,2023-03-08,"$3,987.55"
1807,2023-03-09,"$3,998.66"
1808,2023-03-10,"$3,912.77"
1809,2023-03-13,"$3,835.12"
1810,2023-03-14,"$3,894.01"


In [113]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. INICIALIZAMOS el Prophet y lo ENTRENAMOS pasandole el dataframe
</p>



In [114]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"

m = Prophet(seasonality_mode='multiplicative')

In [115]:
#TAREA: Entrena el modelo (m) con el dataframe de datos (df)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos PREDECIR, en este caso el próximo año.
</p>



In [116]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()

future = m.make_future_dataframe(periods=365)

future.tail()

,ds
2171,2024-03-09
2172,2024-03-10
2173,2024-03-11
2174,2024-03-12
2175,2024-03-13


In [117]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2169,2024-03-07
2170,2024-03-08
2173,2024-03-11
2174,2024-03-12


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [118]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [119]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()     #'yhat' es la columna que representa la predicción del modelo para cada punto en el futuro
                                                                # media, predicción mínima, predicción máxima

,ds,yhat,yhat_lower,yhat_upper
2067,2024-03-07,"$3,142.27","$1,586.64","$4,811.50"
2068,2024-03-08,"$3,134.06","$1,555.85","$4,767.86"
2069,2024-03-11,"$3,112.09","$1,511.75","$4,761.54"
2070,2024-03-12,"$3,105.79","$1,485.90","$4,762.25"
2071,2024-03-13,"$3,097.61","$1,439.64","$4,804.59"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [120]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [121]:
plot_components_plotly(m, forecast)